### Import Modules

<br>

In [14]:
import requests
import json
import time
import itertools
import wget
import os
import pickle
import numpy as np

import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from sklearn.cluster.bicluster import SpectralCoclustering
from sklearn.metrics import precision_recall_curve
import scipy
from ggplot import *

sns.set_style('white')
import tensorflow as tf
import pandas as pd
import keras
from keras.applications.vgg16 import VGG16
from keras.callbacks import LearningRateScheduler
from keras import callbacks
from keras import regularizers as reg
from keras.optimizers import SGD, Adam
from keras.models import Sequential , Model , load_model
from keras.layers import Dense, Dropout, Flatten, Input, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import keras.initializers as init
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import load_model



<br>
### Open the Preprocessed Poster Data

<br>
Features data upload from file and selection of features

<br>

In [2]:
# Open File
x_test_raw = pd.read_csv('Features_test.csv')

# Display available features
for i in range (len(x_test_raw.columns)):
    
    print str(x_test_raw.columns[i])

Unnamed: 0
mR
mG
mB
stdR
stdG
stdB
varR
varG
varB
Dc1
Dc2
Dc3
De1
De2
De3
ID


In [3]:
# Select features
# x_data = x_train_raw.loc[: , ['mR' , 'mG' , 'mB' , 'stdR' , 'stdG' , 'stdB' , 'Dc1' , 'Dc2' , 'Dc3']].values
x_data = x_test_raw.loc[: , ['mR' , 'mG' , 'mB' , 'stdR' , 'stdG' , 'stdB' , 'Dc1' , 'Dc2' , 'Dc3']].values



<br>
### Specify the training/test split

<br>
 - The data will be split into training and test for validation purpose
 - Data augmentation will be subsequently applied for both training and test
 
 <br>

In [4]:
x_test  = x_data

print x_test.shape

(1001, 9)




<br>
### Prepare Data for input

<br>
 - Determine the data axis order according tothe keras configuration
 - Define the input and its attributes
 - Center input data and ensure its type as float 
 - Pre-process labels
 
 <br>
 
** 1 - Arrange Data for Input:**

<br>

In [5]:
# Dfine the data number of columns
dat_cols = x_test.shape[1]



<br>
** 2 - Pre-process Data: **

<br>

In [6]:
# Transform to float
x_test  = x_test.astype('float32')

# Center the data
x_test  -= x_test.mean(0)

# Scale the data
x_test  /= x_test.std(0)

# Display test data attributes
print 'x_test shape:', x_test.shape
print  x_test.shape[0], 'test samples'

x_test shape: (1001, 9)
1001 test samples



<br>
** 3 - Output Data Pre-processing : **

<br>

In [7]:
# Extract Labels from File
y_raw = pd.read_csv('Genres_labels_All_test_cleaned2.csv')

# Split into train and test
y_test  = y_raw.iloc[:, 1:-1].values

# Define the number of classes
num_classes = y_test.shape[1]

print 'number of classes:  ' , num_classes

number of classes:   14



<br>
### Model Import

<br>
 - Import model
 - Perform evaluation
 - Perform prediction
 
<br>
** 1 - Import Model: **

<br>

In [8]:
model = load_model('simple.h5')


<br>
** 2 - Perform Evaluation:**

<br>

In [9]:
loss , overall_acc = model.evaluate(x_test , y_test , batch_size = 64 , verbose = 1)

  64/1001 [>.............................] - ETA: 2s

In [10]:
overall_acc

0.8395177621346015


<br>
** 3 - Perform Prediction: **

<br>

In [11]:
y_predict = model.predict(x_test , batch_size = 16 , verbose = 1)

 912/1001 [==========================>...] - ETA: 0s

In [12]:
# Calculate absolute error
abs_err = np.abs(y_test - y_predict)

# Calculate pprediction proximity measure (1 - absolute error) for visualization
prox = 1.0 - abs_err


<br>
### Vizualization

<br>

In [26]:
# Construct data frame
proxdf = pd.DataFrame(prox , columns = y_raw.columns[1:-1]).iloc[:100 , :]

proxdf.head()

,Action,Adventure,Animation,Comedy,Crime,Drama,Family,History,Horror,Fantasy_Mystery,Romance,Science Fiction,Thriller,Other
0,0.681198,0.247748,0.816986,0.589873,0.693096,0.520056,0.784985,0.168401,0.672990,0.699268,0.723813,0.731744,0.651149,0.784775
1,0.809736,0.876960,0.929093,0.641026,0.212837,0.516778,0.884857,0.935901,0.230493,0.795893,0.842869,0.834815,0.223107,0.907060
2,0.775024,0.895503,0.945519,0.652776,0.820616,0.456247,0.894628,0.947591,0.765867,0.820605,0.847263,0.856577,0.779594,0.907211
3,0.767976,0.851933,0.911654,0.640564,0.795794,0.456213,0.882247,0.930286,0.759077,0.797212,0.825678,0.811427,0.736810,0.880857
4,0.211096,0.171582,0.131512,0.635024,0.725441,0.493276,0.854419,0.917172,0.750500,0.758214,0.774049,0.779979,0.713894,0.872535


<br>
** Plot Accurracy**

<br>


<br>
### Export Results to File:

<br>
** 1 - Export Prediction  : **

<br>

In [28]:
# Construct data frame
y_preddf = pd.DataFrame(y_predict , columns = y_raw.columns[1:-1])

# Export File
y_preddf.to_csv('simplepred.csv')


<br>
** 2 - Export Proximity measure **

<br>

In [29]:


# Write to File
proxdf.to_csv('simple_prox.csv')